In [2]:
import os
os.chdir(r"c:\Users\rohan\Documents\Flutter Projects\saas-subscription-analytics")
os.getcwd()

'c:\\Users\\rohan\\Documents\\Flutter Projects\\saas-subscription-analytics'

In [3]:
import duckdb
con = duckdb.connect("saas.db", read_only=False)

# Drop old tables to avoid duplicates
con.execute("""
DROP TABLE IF EXISTS accounts_clean;
DROP TABLE IF EXISTS subscriptions_clean;
DROP TABLE IF EXISTS churn_clean;
DROP TABLE IF EXISTS feature_usage_clean;
DROP TABLE IF EXISTS support_tickets_clean;
""")

# Load cleaned CSVs
con.execute("""
CREATE TABLE accounts_clean AS 
    SELECT * FROM read_csv_auto('data/processed/accounts_cleaned.csv');

CREATE TABLE subscriptions_clean AS 
    SELECT * FROM read_csv_auto('data/processed/subscriptions_cleaned.csv');

CREATE TABLE churn_clean AS 
    SELECT * FROM read_csv_auto('data/processed/churn_events_cleaned.csv');

CREATE TABLE feature_usage_clean AS 
    SELECT * FROM read_csv_auto('data/processed/feature_usage_cleaned.csv');

CREATE TABLE support_tickets_clean AS 
    SELECT * FROM read_csv_auto('data/processed/support_tickets_cleaned.csv');
""")

# Verify
print(con.execute("SHOW TABLES;").fetchall())


[('accounts',), ('accounts_clean',), ('churn_clean',), ('churn_events',), ('churn_events_clean',), ('feature_usage',), ('feature_usage_clean',), ('subscriptions',), ('subscriptions_clean',), ('support_tickets',), ('support_tickets_clean',)]


In [4]:
query = open("sql/21_signup_retention_analysis.sql").read()
df_signup_retention = con.execute(query).df()
df_signup_retention.head()


,cohort_month,active_month,cohort_users,active_users,retention_rate
0,2023-01-01,2023-01-01,17,2,0.117647
1,2023-01-01,2023-02-01,17,6,0.352941
2,2023-01-01,2023-03-01,17,6,0.352941
3,2023-01-01,2023-04-01,17,8,0.470588
4,2023-01-01,2023-05-01,17,5,0.294118


In [5]:
query = open("sql/22_feature_adoption_analysis.sql").read()
df_feature_cohorts = con.execute(query).df()
df_feature_cohorts.head()


,cohort_month,usage_month,cohort_users,active_users,adoption_rate
0,2023-01-01,2023-01-01,17,16,0.941176
1,2023-01-01,2023-02-01,17,12,0.705882
2,2023-01-01,2023-03-01,17,15,0.882353
3,2023-01-01,2023-04-01,17,16,0.941176
4,2023-01-01,2023-05-01,17,16,0.941176


In [6]:
query = open("sql/23_churn_analysis.sql").read()
df_churn_cohorts = con.execute(query).df()
df_churn_cohorts.head()


,cohort_month,churn_month,cohort_users,churned_users,churn_rate
0,2023-01-01,2023-01-01,17,1,0.058824
1,2023-01-01,2023-04-01,17,1,0.058824
2,2023-01-01,2023-05-01,17,2,0.117647
3,2023-01-01,2023-07-01,17,1,0.058824
4,2023-01-01,2023-09-01,17,1,0.058824


In [7]:
ret_matrix = df_signup_retention.pivot(
    index="cohort_month",
    columns="active_month",
    values="retention_rate"
)
ret_matrix


active_month,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,...,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
cohort_month,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.117647,0.352941,0.352941,0.470588,0.294118,0.235294,0.588235,0.235294,0.235294,0.470588,...,0.352941,0.294118,0.411765,0.176471,0.294118,0.294118,0.470588,0.294118,0.176471,0.352941
2023-02-01,NaN,0.166667,0.388889,0.277778,0.222222,0.333333,0.277778,0.333333,0.166667,0.277778,...,0.444444,0.277778,0.722222,0.277778,0.333333,0.277778,0.388889,0.222222,0.388889,0.388889
2023-03-01,NaN,NaN,0.150000,0.350000,0.300000,0.350000,0.400000,0.450000,0.350000,0.200000,...,0.600000,0.200000,0.100000,0.500000,0.400000,0.300000,0.400000,0.550000,0.400000,0.550000
2023-04-01,NaN,NaN,NaN,0.333333,0.400000,0.333333,0.333333,0.666667,0.400000,0.466667,...,0.533333,0.200000,0.466667,0.400000,0.266667,0.333333,0.466667,0.600000,0.133333,0.466667
2023-05-01,NaN,NaN,NaN,NaN,0.192308,0.461538,0.461538,0.423077,0.384615,0.461538,...,0.307692,0.461538,0.461538,0.230769,0.615385,0.653846,0.538462,0.384615,0.461538,0.423077
2023-06-01,NaN,NaN,NaN,NaN,NaN,0.153846,0.153846,0.615385,0.461538,0.384615,...,0.230769,0.461538,0.307692,0.538462,0.384615,0.461538,0.538462,0.615385,0.307692,0.153846
2023-07-01,NaN,NaN,NaN,NaN,NaN,NaN,0.214286,0.714286,0.214286,0.214286,...,0.642857,0.500000,0.142857,0.285714,0.500000,0.428571,0.500000,0.714286,0.500000,0.500000
2023-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.312500,0.375000,0.375000,...,0.562500,0.375000,0.625000,0.250000,0.687500,0.437500,0.437500,0.562500,0.312500,0.437500
2023-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.260870,0.347826,...,0.652174,0.608696,0.434783,0.434783,0.608696,0.391304,0.565217,0.478261,0.434783,0.608696


In [8]:
adopt_matrix = df_feature_cohorts.pivot(
    index="cohort_month",
    columns="usage_month",
    values="adoption_rate"
)
adopt_matrix


usage_month,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,...,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
cohort_month,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.941176,0.705882,0.882353,0.941176,0.941176,0.764706,0.941176,1.000000,0.823529,0.882353,...,0.823529,0.882353,0.941176,0.705882,0.705882,0.823529,0.705882,0.882353,0.647059,0.823529
2023-02-01,0.777778,0.944444,0.888889,0.888889,0.722222,0.833333,0.777778,0.833333,0.666667,0.944444,...,0.777778,0.666667,0.722222,0.777778,0.833333,0.833333,0.777778,0.722222,0.777778,0.888889
2023-03-01,0.800000,0.700000,0.850000,0.950000,0.900000,0.800000,0.900000,0.800000,0.850000,0.750000,...,0.850000,0.850000,0.850000,0.750000,0.900000,0.850000,0.900000,0.900000,0.750000,0.850000
2023-04-01,0.933333,0.866667,0.933333,0.866667,0.866667,0.866667,0.866667,0.933333,0.733333,0.933333,...,0.933333,0.866667,0.933333,0.933333,0.933333,0.866667,0.800000,1.000000,0.933333,1.000000
2023-05-01,0.884615,0.961538,0.961538,0.923077,0.923077,0.846154,0.884615,0.884615,0.884615,0.923077,...,0.769231,0.846154,0.884615,0.923077,0.884615,0.769231,0.884615,0.923077,0.846154,0.884615
2023-06-01,0.923077,0.846154,0.769231,0.846154,1.000000,0.923077,0.846154,0.923077,0.923077,0.692308,...,0.923077,0.769231,0.846154,0.923077,0.846154,0.846154,0.769231,0.846154,1.000000,0.923077
2023-07-01,0.928571,0.785714,0.928571,0.928571,0.928571,0.857143,1.000000,0.857143,0.785714,0.928571,...,0.928571,0.857143,0.857143,0.928571,1.000000,0.857143,1.000000,0.857143,0.857143,0.928571
2023-08-01,0.750000,0.500000,0.875000,0.875000,0.875000,0.875000,1.000000,0.812500,0.875000,0.937500,...,0.937500,0.937500,0.812500,0.937500,0.875000,0.812500,0.812500,0.937500,0.875000,0.750000
2023-09-01,1.000000,0.826087,1.000000,0.869565,0.782609,0.826087,0.782609,0.869565,0.826087,0.782609,...,0.913043,0.826087,0.956522,0.782609,0.652174,0.826087,0.869565,0.956522,0.782609,0.913043


In [9]:
churn_matrix = df_churn_cohorts.pivot(
    index="cohort_month",
    columns="churn_month",
    values="churn_rate"
)
churn_matrix


churn_month,2023-01-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,...,2024-03-01,2024-04-01,2024-05-01,2024-06-01,2024-07-01,2024-08-01,2024-09-01,2024-10-01,2024-11-01,2024-12-01
cohort_month,,,,,,,,,,,,,,,,,,,,,
2023-01-01,0.058824,NaN,0.058824,0.117647,NaN,0.058824,NaN,0.058824,0.058824,0.117647,...,NaN,NaN,0.058824,0.117647,0.117647,0.058824,NaN,0.058824,NaN,NaN
2023-02-01,NaN,0.111111,0.055556,NaN,0.055556,NaN,NaN,NaN,0.055556,0.166667,...,NaN,0.166667,NaN,0.055556,NaN,0.055556,NaN,0.166667,NaN,NaN
2023-03-01,NaN,0.150000,NaN,0.050000,0.150000,0.050000,0.150000,0.100000,0.100000,NaN,...,NaN,0.100000,NaN,NaN,NaN,0.050000,0.050000,NaN,0.050000,0.100000
2023-04-01,NaN,NaN,0.066667,NaN,0.066667,0.066667,NaN,NaN,0.066667,NaN,...,0.066667,NaN,0.133333,0.200000,0.133333,0.066667,NaN,NaN,0.066667,NaN
2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,0.115385,0.038462,0.038462,0.076923,...,0.038462,0.038462,0.038462,NaN,NaN,0.038462,0.115385,0.115385,0.076923,0.076923
2023-06-01,NaN,NaN,NaN,NaN,NaN,0.076923,NaN,0.076923,NaN,NaN,...,0.076923,0.076923,0.076923,NaN,0.076923,0.076923,0.230769,0.153846,0.230769,NaN
2023-07-01,NaN,NaN,NaN,NaN,NaN,0.142857,NaN,NaN,0.071429,NaN,...,NaN,0.142857,0.071429,0.142857,0.071429,0.142857,NaN,NaN,NaN,0.214286
2023-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.062500,NaN,NaN,...,0.125000,0.062500,0.062500,0.187500,0.125000,NaN,0.187500,NaN,0.125000,NaN
2023-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.086957,0.130435,...,0.173913,0.086957,NaN,0.043478,0.043478,0.173913,0.130435,0.217391,0.086957,0.130435


In [10]:
con.close()